# THREE GEN | SUBNET 17
To demonstrate how SN17 works you need to look at it both from the miner and validator perspective.

## Miner

Prerequisites:
- create wallet,
- register on SN17,
- setup a node,
- install git, conda, pm2.
  
Setup:
- git clone https://github.com/404-Repo/three-gen-subnet.git
- cd three-gen-subnet/generation
- ./setup_env.sh
- pm2 start generation.config.js
- cd ../neurons
- ./setup_env.sh
- pm2 start miner.config.js

This setup will run a generation endpoint locally (responsible for generating 3d assets) and bittensor neuron (responsible for communication within the subnet).

## Miner pseudo-code

Miner fetches tasks from validators (round robin), generate assets and submit results.

In [ ]:
%pip install bittensor

In [ ]:
import bittensor as bt
from common.protocol import PullTask, SubmitResults

wallet = bt.wallet(name="default", hotkey="default")  # validator neuron can be used as well
subtensor = bt.subtensor(network="finney")
metagraph = bt.metagraph(netuid=17, network=subtensor.network, sync=True)                         
dendrite = bt.dendrite(wallet)

# Pulling the task from validator

validator_uid = 0  # validator operated by the subnet owner
synapse = PullTask()
response = await dendrite.call(
            target_axon=metagraph.axons[validator_uid], synapse=synapse, deserialize=False, timeout=12.0
        )
task = response.task

# Generating assets

async with aiohttp.ClientSession(timeout=client_timeout) as session:
    async with session.post("http://127.0.0.1:8094/generate", data={"prompt": task.prompt}) as response:
        assets = await response.text()

# Signing results (needed to verify the results origin when fetching from storage subnet)

submit_time = time.time_ns()
message = f"{submit_time}{task.prompt}{metagraph.hotkeys[validator_uid]}{wallet.hotkey.ss58_address}"
signature = base64.b64encode(dendrite.keypair.sign(message)).decode(encoding="utf-8")

# Submitting results

synapse = SubmitResults(task=task, results=assets, submit_time=submit_time, signature=signature)
response = await dendrite.call(
    target_axon=metagraph.axons[validator_uid],
    synapse=synapse,
    deserialize=False,
    timeout=300.0,
)

# Printing feedback

bt.logging.debug(f"Feedback received. Prompt: {response.task.prompt}. Score: {response.feedback.task_fidelity_score}")
bt.logging.debug(
    f"Average score: {response.feedback.average_fidelity_score}. "
    f"Accepted results (last 8h): {response.feedback.generations_within_8_hours}. "
    f"Reward: {response.feedback.current_miner_reward}."
)

This will print:

> Feedback received. Prompt: iridescent ice cube tray. Score: 1.0

> Average score: 0.8543000416514985. Accepted results (last 8h): 39. Reward: 33.31770162440844.

Meaning that results for the task with the prompt `iridescent ice cube tray` have been accepted. The fidelity score for the current generation is 1.0.
EMA of the all fidelity scores is 0.85 and total number of accepted results with the score >0.75 during the last 8 hours is 39. Total miner reward is 33.32 (fidelity score * number of accepted results). Normalized miner reward is used as a weight.

There three possible outcomes for the fidelity score.
1.0 - CLIP distance between a prompt and renders is >= 0.8.
0.75 - CLIP distance between a prompt and renders is >= 0.6 and < 0.8.
0 - CLIP distance between a prompt and renders is < 0.6.

Results with fidelity score 0 are not accepted and have no effect on average fidelity score.

In the future, with the advance of the AI models, 0.6 and 0.8 threshold will be increased.


## Validator

Prerequisites:
- create wallet,
- register on SN17,
- setup a node,
- install git, conda, pm2.
  
Setup:
- git clone https://github.com/404-Repo/three-gen-subnet.git
- cd three-gen-subnet/validation
- ./setup_env.sh
- pm2 start validatoin.config.js
- cd ../neurons
- ./setup_env.sh
- pm2 start validator.config.js

This setup will run validation endpoint locally (responsible for scoring generated 3d assets) and bittensor neuron (responsible for communication within the subnet).

## Validator pseudo-code

Validators receive organic request via public API or use synthetic task dataset if no organic request registered. Submitted results are evaluated and an 8-hours window of submitted results is tracked.

In [ ]:
import bittensor as bt
from common.protocol import PullTask, SubmitResults

wallet = bt.wallet(name="default", hotkey="default")
subtensor = bt.subtensor(network="finney")
metagraph = bt.metagraph(netuid=17, network=subtensor.network, sync=True)                         

axon = bt.axon(wallet=wallet, config=self.config)
self.axon.attach(
    forward_fn=pull_task
).attach(
    forward_fn=submit_results
)

def pull_task(synapse: PullTask) -> PullTask:
    organic_task = self.task_registry.get_next_task(synapse.dendrite.hotkey)
    if organic_task is not None:
        task = Task(id=organic_task.id, prompt=organic_task.prompt)
    else:
        task = Task(prompt=self.dataset.get_random_prompt())

    synapse.task = task
    synapse.submit_before = int(time.time()) + self.config.generation.task_timeout
    synapse.version = NEURONS_VERSION
    return synapse

async def submit_results(synapse: SubmitResults) -> SubmitResults:
    uid = get_neuron_uid(synapse.dendrite.hotkey)
    miner = miners[uid]
    
    if not verify_results_signature(synapse):
        return add_feedback(synapse, miner)

    async with aiohttp.ClientSession() as session:
        async with session.post("http://127.0.0.1:8093", json={"prompt": synapse.task.prompts, "data": synapse.results}) as response:
            results = await response.json()
            validation_score = float(results["score"])

    if validation_score >= 0.8:
        fidelity_score = 1
    elif validation_score >= 0.6:
        fidelity_score = 0.75
    else:
        fidelity_score = 0

    if fidelity_score == 0:
        return add_feedback(synapse, miner)

    storage.store(synapse)  # storing to SN21

    miner.add_observation(fidelity_score)

    task_registry.complete_task(synapse.task.id, synapse.dendrite.hotkey, synapse.results, validation_score)

    return add_feedback(synapse, miner, fidelity_score=fidelity_score)

def add_feedback(
    synapse: SubmitResults,
    miner: MinerData,
    fidelity_score: float = 0.0,
    current_time: int | None = None,
) -> SubmitResults:
    if current_time is None:
        current_time = int(time.time())
    reward = miner.fidelity_score * len(miner.observations)
    synapse.feedback = Feedback(
        task_fidelity_score=fidelity_score,
        average_fidelity_score=miner.fidelity_score,
        generations_within_8_hours=len(miner.observations),
        current_miner_reward=reward,
    )
    synapse.cooldown_until = current_time + self.config.generation.task_cooldown
    return synapse

## Miner incentive
There is a clear path on how to increase the incentive.
- Running higher tier GPU or using multiple generation endpoints to submit more results, will increase the miner reward.
- Train or replace the 3D model to generate acceptable results for all prompts.
- Train or replace the 3D model to generate results with higher fidelity score.

With the advance of 3D generation models, quality criteria for generated images will be increased.  

## Validation algorithm

In [ ]:
import numpy as np
from transformers import CLIPProcessor, CLIPModel

def render_images(miner_result: str) -> list[np.ndarray]:
    """ Function for rendering multiple view of the input geometry
    Args:
        miner_result: encoded buffer with data that contains generated geometry provided by the miner
    Returns: a list with rendered images
    """
    geometry = unpack(miner_result)
    orbitcam = OrbitCamera()
    images = []
    for azimuth_angle in range(0, 360, 10):
        # min_ver = -20, max_ver = 20 degrees
        elevation_angle = np.random.randint(min_ver, max_ver)
        # get orbit camera transformation matrix 4x4 = [R | T]
        pose = orbit_camera(angle)
        camera = BasicCamera(pose)
        image = renderer.render(camera, geometry)
        images.append(image)
    return images


def score(prompt: str, images: list[np.ndarray]) -> float
    """ Function for scoring the the result of miner's work using CLIP model
    Args:
        prompt: string with input prompt that was used for generating the input geometry
        images: a list with rendered images of the input geometry
    Returns: a single score between 0 and 1 that identifies how far the generated geometry from the 
             used prompt for its generation. 
    """
    # preloading CLIP model
    model = CLIPModel.from_pretrained(scoring_model)
    processor = CLIPProcessor.from_pretrained(scoring_model)

    # add prompts (always false) against which input prompt
    # will be compared
    negative_prompts = [
            "empty",
            "nothing",
            "false",
            "wrong",
            "negative",
            "not quite right",
        ]
    negative_prompts.append(prompt)

    scores = []
    for img in images:
        inputs = processor(prompts, img)
        results = model(**inputs)
        # we take the score for the last prompt in negative prompts
        # that will be input prompt
        score = logits_per_image.softmax(dim=1)[0][-1]
        scores.append(score)
        
    return np.mean(scores)


def validate(prompt: str, miner_result: str):
    """ Function for computing the validation score
    Args:
        prompt string with input prompt that was used for generating the input geometry
        miner_result: encoded buffer with data that contains generated geometry provided by the miner
    Returns: a float value (validation score) between 0 and 1
    """
    images = render_images(miner_result)
    validation_score = score(prompt, miner_result)
    return validation_score 